In [1]:
train=spark.read.option('header','True').option('inferSchema','True').csv(r'C:\spark_projects\bigmart\Train_UWu5bXk.csv')

#train.count()
#train.head()

In [2]:
test=spark.read.option('header','True').option('inferSchema','True').csv(r'C:\spark_projects\bigmart\Test_u94Q5KV.csv')

In [3]:
train.printSchema

<bound method DataFrame.printSchema of DataFrame[Item_Identifier: string, Item_Weight: double, Item_Fat_Content: string, Item_Visibility: double, Item_Type: string, Item_MRP: double, Outlet_Identifier: string, Outlet_Establishment_Year: int, Outlet_Size: string, Outlet_Location_Type: string, Outlet_Type: string, Item_Outlet_Sales: double]>

In [4]:
train.show(3)

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|      Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         443.4228|
|          FDN1

In [5]:
from pyspark.sql.functions import col,sum
train.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in train.columns)).show()

+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|              0|       1463|               0|              0|        0|       0|                0|                        0|       2410|                   0|          0|                0|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+



In [6]:
from pyspark.sql.functions import col,sum
test.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in test.columns)).show()

+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+
|              0|        976|               0|              0|        0|       0|                0|                        0|       1606|                   0|          0|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+



#Fill in missing data with Imputer 

In [7]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=["Item_Weight"], 
    outputCols=["Weight_imputed"]
)
df_change=imputer.fit(train).transform(train)

In [8]:
tester=imputer.fit(test).transform(test)

In [9]:
test.crosstab('Outlet_Size','Outlet_Type').show()

+-----------------------+-------------+-----------------+-----------------+-----------------+
|Outlet_Size_Outlet_Type|Grocery Store|Supermarket Type1|Supermarket Type2|Supermarket Type3|
+-----------------------+-------------+-----------------+-----------------+-----------------+
|                 Medium|            0|              620|              618|              624|
|                   High|            0|              621|                0|                0|
|                  Small|          352|             1240|                0|                0|
|                   null|          370|             1236|                0|                0|
+-----------------------+-------------+-----------------+-----------------+-----------------+



In [10]:
tester.crosstab('Outlet_Size','Outlet_Location_Type').show()

+--------------------------------+------+------+------+
|Outlet_Size_Outlet_Location_Type|Tier 1|Tier 2|Tier 3|
+--------------------------------+------+------+------+
|                          Medium|   620|     0|  1242|
|                            High|     0|     0|   621|
|                           Small|   972|   620|     0|
|                            null|     0|  1236|   370|
+--------------------------------+------+------+------+



In [11]:
df_change.crosstab('Outlet_Identifier','Outlet_Size').show()

+-----------------------------+----+------+-----+----+
|Outlet_Identifier_Outlet_Size|High|Medium|Small|null|
+-----------------------------+----+------+-----+----+
|                       OUT045|   0|     0|    0| 929|
|                       OUT013| 932|     0|    0|   0|
|                       OUT049|   0|   930|    0|   0|
|                       OUT046|   0|     0|  930|   0|
|                       OUT027|   0|   935|    0|   0|
|                       OUT035|   0|     0|  930|   0|
|                       OUT017|   0|     0|    0| 926|
|                       OUT010|   0|     0|    0| 555|
|                       OUT019|   0|     0|  528|   0|
|                       OUT018|   0|   928|    0|   0|
+-----------------------------+----+------+-----+----+



In [12]:
df_change=df_change.fillna('Small','Outlet_Size')
       

In [13]:
tester=tester.fillna('Small','Outlet_Size')

In [14]:
from pyspark.sql.functions import col,sum
tester.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in tester.columns)).show()

+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+--------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Weight_imputed|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+--------------+
|              0|        976|               0|              0|        0|       0|                0|                        0|          0|                   0|          0|             0|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+--------------+



In [15]:
tester=tester.drop('Item_Weight')
tester=tester.withColumnRenamed('Weight_imputed','Item_Weight')

In [16]:
df_change=df_change.drop('Item_Weight')
df_change=df_change.withColumnRenamed('Weight_imputed','Item_Weight')

In [18]:
menaMRP=df_change.groupBy('Item_Identifier').mean('Item_MRP')
menaMRP=menaMRP.withColumnRenamed('Item_Identifier','Dup')
#menaMRP.show(10)

In [19]:
from pyspark.sql.functions import *
trainer=df_change.join(menaMRP, (df_change.Item_Identifier==menaMRP.Dup))

trainer.show(2)
#test=test.drop('Product_Category_2','Product_Category_3','Dup','User_Dup')

+---------------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+-----------+-----+-------------+
|Item_Identifier|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|Item_Weight|  Dup|avg(Item_MRP)|
+---------------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+-----------+-----+-------------+
|          FDA15|         Low Fat|    0.016047301|      Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|        9.3|FDA15|     249.4967|
|          DRC01|         Regular|    0.019278216|Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|     

In [20]:
trainer=trainer.withColumnRenamed('avg(Item_MRP)','meanMRP')


In [21]:
trainer=trainer.withColumn('logmeanMRP',log(trainer.meanMRP))


In [26]:
trainer=trainer.withColumn('smallgrocery',(trainer.Outlet_Size=='Small')&(trainer.Outlet_Type=='Grocery store'))
trainer=trainer.withColumn('smallsm1',(trainer.Outlet_Size=='Small')&(trainer.Outlet_Type=='Supermarket Type1'))
trainer=trainer.withColumn('medsm1',(trainer.Outlet_Size=='Medium')&(trainer.Outlet_Type=='Supermarket Type1'))
trainer=trainer.withColumn('highsm1',(trainer.Outlet_Size=='High')&(trainer.Outlet_Type=='Supermarket Type1'))
trainer=trainer.withColumn('medsm2',(trainer.Outlet_Size=='Medium')&((trainer.Outlet_Type=='Supermarket Type2')|(trainer.Outlet_Type=='Supermarket Type3')))
trainer=trainer.withColumn('storeage',(2013-trainer.Outlet_Establishment_Year))

In [27]:
trainer=trainer.withColumn('Nosold',(trainer.Outlet_Size=='Medium')&(trainer.Outlet_Type=='Supermarket Type3'))

In [28]:
tester=tester.join(menaMRP,tester.Item_Identifier==menaMRP.Dup)
tester=tester.withColumnRenamed('avg(Item_MRP)','meanMRP')



In [29]:
tester=tester.withColumn('logmeanMRP',log(tester.meanMRP))

In [30]:
tester=tester.withColumn('smallgrocery',(tester.Outlet_Size=='Small')&(tester.Outlet_Type=='Grocery store'))
tester=tester.withColumn('smallsm1',(tester.Outlet_Size=='Small')&(tester.Outlet_Type=='Supermarket Type1'))
tester=tester.withColumn('medsm1',(tester.Outlet_Size=='Medium')&(tester.Outlet_Type=='Supermarket Type1'))
tester=tester.withColumn('highsm1',(tester.Outlet_Size=='High')&(tester.Outlet_Type=='Supermarket Type1'))
tester=tester.withColumn('medsm2',(tester.Outlet_Size=='Medium')&((tester.Outlet_Type=='Supermarket Type2')|(tester.Outlet_Type=='Supermarket Type3')))
tester=tester.withColumn('storeage',(2013-tester.Outlet_Establishment_Year))

In [ ]:
trainer.show(2)

In [ ]:
print(df_change.select('Outlet_Identifier').distinct().count())
print(df_change.select('Item_Identifier').distinct().count())
print(df_change.select('Item_MRP').distinct().count())

In [ ]:
print(test.select('Outlet_Identifier').distinct().count())
print(test.select('Item_Identifier').distinct().count())

In [ ]:
diff_cat_in_train_test=test.select('Item_Identifier').subtract(df_change.select('Item_Identifier'))
diff_cat_in_train_test.distinct().count()# For distict count


In [ ]:
df_change.schema

In [31]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from pyspark.sql import *
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
categorical_columns = ['Item_Type','Outlet_Identifier']
#categorical_columns = ['Outlet_Identifier']

##=== build stages ======
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='stringindexed_' + c) for c in categorical_columns]

onehotencoder_stages = [OneHotEncoder(inputCol='stringindexed_' + c, outputCol='onehotencoded_' + c) for c in categorical_columns]
selected_columns = ['onehotencoded_' + c for c in categorical_columns] + ['Item_MRP','smallgrocery','smallsm1','medsm1','highsm1','medsm2','storeage']
#selected_columns = ['onehotencoded_' + c for c in categorical_columns] + ['meanMRP']
vectorassembler_stages = [VectorAssembler(inputCols=selected_columns, outputCol='features')]
all_stages = stringindexer_stages + onehotencoder_stages+vectorassembler_stages





In [32]:
pipeline = Pipeline(stages=all_stages)

## fit pipeline model
df_features = pipeline.fit(trainer).transform(trainer)

In [33]:
testerfinal = pipeline.fit(tester).transform(tester)

In [35]:
import pyspark.ml.evaluation as ev
evaluator = ev.RegressionEvaluator(
    predictionCol='prediction', 
    labelCol='Item_Outlet_Sales')

In [36]:
#training
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pyspark.ml.regression as rg
linalg= rg.LinearRegression(labelCol='Item_Outlet_Sales')

paramGrid = ParamGridBuilder()\
    .addGrid(linalg.maxIter,[10,100])\
    .build()
    
crossval = CrossValidator(
    estimator=linalg, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=6)

trainresult=crossval.fit(df_features)
# lrmodel = linalg.fit(df_features)
# lrtmodel = lrmodel.transform(df_features)

In [37]:
trainresult.avgMetrics

[1132.1801123443342, 1132.6929980572781]

In [ ]:
result.show(3)

In [ ]:
result = lrmodel.transform(tester)

In [ ]:
from pyspark.sql.functions import col,sum
submission =testresult.select(col("Item_Identifier"),col("Outlet_Identifier"),col("prediction"))
final=submission.withColumnRenamed('prediction','Item_Outlet_Sales')
final.show(2)


In [ ]:
final.write.csv('Bigmartsale9.csv',header=True)

In [ ]:
from pyspark.sql.functions import col,sum
submission=result.select(col("Product_ID"),col("prediction").cast('int'))





In [ ]:
submission.head()

In [38]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="Item_Outlet_Sales")
 
model = rf.fit(df_features)
rfpredictions = model.transform(df_features)
 
#tpredictions = model.transform(test_data)

In [39]:
print(evaluator.evaluate(rfpredictions, 
    {evaluator.metricName: 'rmse'}))

1159.5685004918219


In [ ]:
result = model.transform(testerfinal)

In [40]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(labelCol="Item_Outlet_Sales")
 
gbtmodel = gbt.fit(df_features)
gbtpredictions = gbtmodel.transform(df_features)

In [ ]:
result=gbtmodel.transform(testerfinal)
#result.show(2)

In [41]:
print(evaluator.evaluate(gbtpredictions, 
    {evaluator.metricName: 'rmse'}))

1039.8295419144301


In [ ]:
print(evaluator.evaluate(lrtmodel, 
    {evaluator.metricName: 'rmse'}))

In [ ]:
submission =lrtmodel.select(col('User_ID'),col("Purchase"),col("prediction").cast('int'))

In [ ]:
submission.show(20)

In [42]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(labelCol="Item_Outlet_Sales",featuresCol="features")
paramGrid = ParamGridBuilder()\
    .addGrid(gbt.maxDepth,[2,5])\
    .addGrid(gbt.maxIter,[10,100])\
    .build()
    
crossval = CrossValidator(
    estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=6)
result=crossval.fit(df_features)

In [43]:
result.avgMetrics

[1184.2675891963736,
 1085.5291745345496,
 1091.0529378820088,
 1145.9881312091436]

In [ ]:
trainresult=result.transform(df_features)

In [ ]:
print(evaluator.evaluate(trainresult, 
    {evaluator.metricName: 'rmse'}))

In [ ]:
testresult=result.transform(testerfinal)

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="Item_Outlet_Sales")
 
paramGrid = ParamGridBuilder()\
    .addGrid(rf.maxDepth,[2,5])\
    .addGrid(rf.numTrees,[10,100])\
    .build()
    
crossval = CrossValidator(
    estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=6)


rfmodel = crossval.fit(df_features)

In [ ]:
rfmodel.avgMetrics